In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
Nlat = 100
Nlong = 200

lat_vect = np.linspace(-np.pi/2, np.pi/2, Nlat)
long_vect = np.linspace(-np.pi, np.pi, Nlong)

xy_ones = np.ones([Nlat, Nlong])
lat_arr = xy_ones * np.expand_dims(lat_vect, 1)
long_arr = xy_ones * np.expand_dims(long_vect, 0)

In [ ]:
plt.matshow(lat_arr)

In [ ]:
plt.matshow(long_arr)

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    return np.arcsin(np.sqrt(a))

In [ ]:
def dist_map(lon, lat, long_arr, lat_arr):
    dist_arr = haversine(lon, lat, long_arr, lat_arr)
    return dist_arr

In [ ]:
plt.matshow(dist_map(0.5, 1, long_arr, lat_arr))

In [ ]:
from scipy.interpolate import RectBivariateSpline, RegularGridInterpolator

In [ ]:
def ampliate(arr, N_hd = 1000):
    n_y, n_x = arr.shape
    lat_vec = np.linspace(-np.pi/2, np.pi/2, n_y)
    long_vec = np.linspace(-np.pi/2, np.pi/2, n_x)
    lat_vect_hd = np.linspace(-np.pi/2, np.pi/2, N_hd)
    long_vect_hd = np.linspace(-np.pi, np.pi, N_hd*2)
    xy_ones_hd = np.ones([N_hd, N_hd*2])
    lat_arr_hd = xy_ones_hd * np.expand_dims(lat_vect_hd, 1)
    long_arr_hd = xy_ones_hd * np.expand_dims(long_vect_hd, 0)
    
    _f = RectBivariateSpline(lat_vect, long_vect, arr, kx = 2, ky = 2)
    newarr = _f(lat_vect_hd, long_vect_hd)
    return newarr

In [ ]:
N_plates = 10

centers = []
for i in range(N_plates):
    centers.append(tuple((np.random.rand(2)*2-1)*np.array([np.pi, np.pi/2])))

cent_dist = np.zeros([Nlat, Nlong, N_plates])
for i in range(N_plates):
    cent_dist[:,:,i] = dist_map(*centers[i], long_arr, lat_arr)
    #plt.matshow(cent_dist[:,:,i])

In [ ]:
N_hd = 1000

for i in range(N_plates):
    cent_dist_hd[:,:,i] = ampliate(cent_dist[:,:,i], N_hd)

In [ ]:
plate_map = np.argmin(cent_dist_hd, axis=2)

In [ ]:
plt.figure(figsize=[18,9])
plt.imshow(plate_map)

In [ ]:
import opensimplex

In [ ]:
def noise_sphere(long_arr, lat_arr, N = 200, scale = 2):
    vector = np.linspace(-scale*np.pi, scale*np.pi, N)
    opensimplex.random_seed()
    noise_3d = opensimplex.noise3array(vector, vector,vector)
    interp = RegularGridInterpolator((vector, vector, vector), noise_3d)
    x_sph = np.cos(lat_arr)*np.cos(long_arr)*scale
    y_sph = np.cos(lat_arr)*np.sin(long_arr)*scale
    z_sph = np.sin(lat_arr)*scale
    noise_interp = interp((x_sph, y_sph, z_sph))
    return noise_interp

In [ ]:
_noise_sph = noise_sphere(long_arr, lat_arr, N = 200, scale = 8)
plt.figure(figsize=[18,9])
plt.imshow(_noise_sph)

In [ ]:
plt.figure(figsize=[18,9])
plt.imshow(ampliate(_noise_sph, 1000))

In [ ]:
def fract_noise(x, y, level = 3):
    opensimplex.random_seed()
    base = opensimplex.noise2array(x, y)
    for i in range(level-1):
        j = i+2
        opensimplex.random_seed()
        base += opensimplex.noise2array(x*j, y*j)/j
    minz = np.min(base)
    maxz = np.max(base)
    dz = maxz-minz
    base = (base - minz)/dz
    return base

In [ ]:
def fract_noise_sphere(N_hd, level = 3, minlevel = 2, N = 100):
    
    lat_vec = np.linspace(-np.pi/2, np.pi/2, N)
    long_vec = np.linspace(-np.pi/2, np.pi/2, 2*N)
    
    xy_ones = np.ones([N, 2*N])
    lat_arr = xy_ones * np.expand_dims(lat_vect, 1)
    long_arr = xy_ones * np.expand_dims(long_vect, 0)
    
    base = noise_sphere(long_arr, lat_arr, 2*N, scale = minlevel)
    for i in range(minlevel+1, level+1):
        base += noise_sphere(long_arr, lat_arr, 2*N, scale = i)
    minz = np.min(base)
    maxz = np.max(base)
    dz = maxz-minz
    base = (base - minz)/dz
    return ampliate(base, N_hd)

In [ ]:
plt.figure(figsize=[18,9])
plt.imshow(fract_noise_sphere(1000,7))

In [ ]:
def noisy_plates(N_plates = 10, N_hd = 1000, rand_k = 0.5, level = 3):

    centers = []
    for i in range(N_plates):
        centers.append(tuple((np.random.rand(2)*2-1)*np.array([np.pi, np.pi/2])))

    cent_dist = np.zeros([Nlat, Nlong, N_plates])
    for i in range(N_plates):
        cent_dist[:,:,i] = dist_map(*centers[i], long_arr, lat_arr) * (0.5 + rand_k *fract_noise_sphere(Nlat,level))
        
        
    cent_dist_hd = np.zeros([N_hd, N_hd*2, N_plates])
    for i in range(N_plates):
        cent_dist_hd[:,:,i] = ampliate(cent_dist[:,:,i], N_hd)
        
    plate_map = np.argmin(cent_dist_hd, axis=2)
    return plate_map, np.array(centers)

In [ ]:
plates, centers = noisy_plates(rand_k=0.3, level = 5)

In [ ]:
centers_mov = np.random.rand(10, 2)
centers_mov[:,0] = centers_mov[:,0]*2*np.pi
c_mov_uv = np.zeros_like(centers_mov)
c_mov_uv[:,0] = centers_mov[:,1] * np.cos(centers_mov[:,0])
c_mov_uv[:,1] = centers_mov[:,1] * np.sin(centers_mov[:,0])

In [ ]:
plt.figure(figsize=[18,9])
plt.imshow(plates, origin = 'lower', extent = (-np.pi, np.pi, -np.pi/2, np.pi/2))
plt.scatter(centers[:,0],centers[:,1], c = 'red')
plt.quiver(centers[:,0],centers[:,1],c_mov_uv[:,0],c_mov_uv[:,1])

In [ ]:
c_mov_cart = np.zeros([10,3])
_u = c_mov_uv[:,0]
_v = c_mov_uv[:,1]
_lat = centers[:,1]
_long = centers[:,0]
c_mov_cart[:,0] = - _u * np.sin(_long) - _v * np.sin(_lat)*np.cos(_long)
c_mov_cart[:,1] =   _u * np.cos(_long) - _v * np.sin(_lat)*np.sin(_long)
c_mov_cart[:,2] =   _v * np.cos(_lat)

In [ ]:
centers_cart = np.zeros([10,3])
centers_cart[:,0] = np.cos(_lat)*np.cos(_long)
centers_cart[:,1] = np.cos(_lat)*np.sin(_long)
centers_cart[:,2] = np.sin(_lat)

In [ ]:
omegas = np.cross(c_mov_cart,centers_cart)

In [ ]:
cart_arr = np.zeros([100,200,3])
cart_arr[:,:,0] = np.cos(lat_arr)*np.cos(long_arr)
cart_arr[:,:,1] = np.cos(lat_arr)*np.sin(long_arr)
cart_arr[:,:,2] = np.sin(lat_arr)

In [ ]:
surf_speeds_map_cart = np.zeros([100 ,200, 10, 3 ])
for ii in range(10):
    surf_speeds_map_cart[:,:,ii,:] = np.cross(cart_arr, omegas[ii,:])

In [ ]:
rot_matrix = np.zeros([100,200,3,3])
rot_matrix[:,:,0,0]= -np.sin(long_arr)
rot_matrix[:,:,0,1]= -np.sin(lat_arr) * np.cos(long_arr)
rot_matrix[:,:,0,2]= np.cos(lat_arr) * np.cos(long_arr)
rot_matrix[:,:,1,0]= np.cos(long_arr)
rot_matrix[:,:,1,1]= -np.sin(lat_arr) * np.sin(long_arr)
rot_matrix[:,:,1,2]= np.cos(lat_arr) * np.sin(long_arr)
rot_matrix[:,:,2,0]= 0
rot_matrix[:,:,2,1]= np.cos(lat_arr)
rot_matrix[:,:,2,2]= np.sin(lat_arr)

In [ ]:
surf_speeds_map = np.matmul(surf_speeds_map_cart, rot_matrix)

In [ ]:
speedmap = np.zeros([100,200,2])
reduced_plate = plates[::10,::10]
for i in range(100):
    for j in range(200):
        ind_map = reduced_plate[i,j]
        speedmap[i,j,:] = surf_speeds_map[i,j,ind_map,:2]

In [ ]:
plt.figure(figsize=[18,9])
plt.imshow(plates, origin = 'lower', extent = (-np.pi, np.pi, -np.pi/2, np.pi/2))
plt.scatter(centers[:,0],centers[:,1], c = 'red')
plt.quiver(centers[:,0],centers[:,1],c_mov_uv[:,0],c_mov_uv[:,1], alpha = 0.5)
plt.quiver(long_arr[::5,::5] ,lat_arr[::5,::5],speedmap[::5,::5,0],speedmap[::5,::5,1])